In [4]:
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
from torchvision import transforms
from importlib import reload
import util 
reload(util)
from util import get_CIFAR10_loaders, train_model, plot_accuracy, plot_training_loss

# CNNs - VGG16

![LeNet-5](img/VGG16-architecture.png)

*Source: https://www.researchgate.net/publication/322512435_Automatic_localization_of_casting_defects_with_convolutional_neural_networks*

In [8]:
class VGG16(torch.nn.Module):
    def __init__(self, num_classes):
        super(VGG16, self).__init__()
        self.conv_1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_5 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(512, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.conv_1(x)
        x = self.conv_2(x)
        x = self.conv_3(x)
        x = self.conv_4(x)
        x = self.conv_5(x)
        x = x.view(x.size(0), 512)
        logits = self.classifier(x)
        return logits

## VGG16 on CIFAR10

In [9]:
# Architecture
num_classes = 10

# Hyperparameters
learning_rate = 0.001
batch_size = 256
num_epochs = 15
random_seed = 0

# Data transforms and loaders
train_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                       transforms.ToTensor()])

test_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                      transforms.ToTensor()])

train_loader, valid_loader, test_loader = get_CIFAR10_loaders(
    batch_size=batch_size,
    train_transform = train_transforms,
    test_transform = test_transforms,
    num_workers=2
)

# GPU setup
DEVICE = "cpu"
if torch.cuda.is_available():
    DEVICE = "cuda:0"
    torch.backends.cudtorch.nn.benchmark = False
    torch.backends.cudtorch.nn.deterministic = True

Files already downloaded and verified


In [10]:
import time
torch.manual_seed(random_seed)
model = VGG16(num_classes)
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()

minibatch_loss, train_acc, valid_acc = train_model(
    model=model, 
    optimizer=optimizer, 
    loss_fn=loss_fn, 
    train_loader=train_loader, 
    valid_loader=valid_loader, 
    test_loader=test_loader, 
    num_epochs=num_epochs, 
    batch_size=batch_size,
    device=DEVICE
)

In [ ]:
plot_accuracy(train_acc, valid_acc)

In [ ]:
plot_training_loss(minibatch_loss)